# **Sentiment Analysis on Reddit**

In [9]:
import praw
import pandas as pd
import numpy as np
import nltk
import fuzzywuzzy as fuzz
import requests
from bs4 import BeautifulSoup

import bot1

reddit = praw.Reddit(client_id=bot1.app_id, client_secret=bot1.app_secret, user_agent=bot1.app_ua)
sub = 'pcgaming'
subreddit = reddit.subreddit(sub)
topic_list = []
num_posts = 5

def get_topic(topic_list):
    topic = ''

    return topic

def score_comments(url):
    pass

for post in subreddit.hot(limit=num_posts):
    url = 'reddit.com/r/' + sub + '/comments/' + post.id
    score_comments(url)

def read_wiki_table(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table',{'class':"wikitable sortable"})
    df = pd.read_html(str(table))
    df = pd.DataFrame(df[0])
    if 'Name' not in df:
        if 'Title' in df:
            df.rename(columns={'Title': 'Name'})
        elif 'Game' in df:
            df.rename(columns={'Game': 'Name'})
    return df

def get_games():
    site_list = [
        'https://en.wikipedia.org/wiki/List_of_massively_multiplayer_online_role-playing_games',
        'https://en.wikipedia.org/wiki/List_of_PC_games_(Numerical)',
        'https://en.wikipedia.org/wiki/List_of_most-played_video_games_by_player_count',
        'https://en.wikipedia.org/wiki/List_of_best-selling_PC_games'
    ]
    games = pd.DataFrame()
    from string import ascii_uppercase
    for url in site_list:
        games = pd.concat([games, read_wiki_table(url)])
    for c in ascii_uppercase:
        wikiurl = "https://en.wikipedia.org/wiki/List_of_PC_games_(" + c + ")"
        games = pd.concat([games, read_wiki_table(wikiurl)])
    
    games['Name'].fillna(games['Title'], inplace=True)
    games.drop(['Title'], axis=1, inplace=True)
    games.drop_duplicates(subset=['Name'], inplace=True)
    titles = games['Name']
    return titles

topics = get_games()


0            The 4th Coming
1         AdventureQuest 3D
2     AdventureQuest Worlds
3              Age of Conan
4              Age of Wulin
              ...          
25               Zup! Arena
26                   Zup! X
27                Zup! Zero
28              Zup! Zero 2
29                      ZZT
Name: Name, Length: 1273, dtype: object
